In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("VisualizandoDataset").getOrCreate()

In [0]:
# Caminho do dataset
caminho_dataset = "dbfs:/FileStore/tables/financial_transactions.csv"

#Visualizando o dataset
df_financial = spark.read.csv(caminho_dataset, header=True, inferSchema=True)
df_financial.show()




+--------------+----------+-----------+------+-----------+-----------------+
|transaction_id|      date|customer_id|amount|   category|   payment_method|
+--------------+----------+-----------+------+-----------+-----------------+
|             1|2024-02-10|       1042| 97.54|   Educação|         Dinheiro|
|             2|2024-01-08|       1017|457.19|      Saúde| Cartão de Débito|
|             3|2024-01-02|       1044|436.55|   Educação|         Dinheiro|
|             4|2024-02-17|       1043|156.24| Transporte| Cartão de Débito|
|             5|2024-01-18|       1041|323.09| Transporte|              Pix|
|             6|2024-01-16|       1004| 308.4|      Lazer|              Pix|
|             7|2024-01-15|       1038| 84.89| Transporte|Cartão de Crédito|
|             8|2024-01-09|       1040|383.63|Alimentação|Cartão de Crédito|
|             9|2024-02-17|       1010| 274.3|      Saúde|Cartão de Crédito|
|            10|2024-01-07|       1034|391.53|      Saúde| Cartão de Débito|

In [0]:
# Visualização dos tipos de dados 
df_financial.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- category: string (nullable = true)
 |-- payment_method: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, sum as _sum

# Verifica se as colunas tem números nulos e quantidade desses números
df_financial.select([_sum(col(c).isNull().cast("int")).alias(c) for c in df_financial.columns]).show()

+--------------+----+-----------+------+--------+--------------+
|transaction_id|date|customer_id|amount|category|payment_method|
+--------------+----+-----------+------+--------+--------------+
|             0|   0|          0|     0|       0|             0|
+--------------+----+-----------+------+--------+--------------+



In [0]:
# Verifica se há transações com o amount negativo
df_financial.filter(col("amount") < 0).show()

+--------------+----+-----------+------+--------+--------------+
|transaction_id|date|customer_id|amount|category|payment_method|
+--------------+----+-----------+------+--------+--------------+
+--------------+----+-----------+------+--------+--------------+



In [0]:
from pyspark.sql.functions import count

# Verificar duplicatas em todas as colunaas
df_financial.groupBy(df_financial.columns).count().filter("count > 1").show()

+--------------+----+-----------+------+--------+--------------+-----+
|transaction_id|date|customer_id|amount|category|payment_method|count|
+--------------+----+-----------+------+--------+--------------+-----+
+--------------+----+-----------+------+--------+--------------+-----+



In [0]:
from pyspark.sql.functions import round

# Calcula o gasto total por clientes agrupando os dados pelo customer_id
df_total_gasto = df_financial.groupBy("customer_id").agg(round(_sum("amount"), 2).alias("total_gasto"))
df_total_gasto.show()

+-----------+-----------+
|customer_id|total_gasto|
+-----------+-----------+
|       1025|     734.46|
|       1016|     809.11|
|       1005|     241.61|
|       1031|     787.43|
|       1034|     715.98|
|       1046|    1420.77|
|       1008|     250.49|
|       1047|     244.94|
|       1021|     463.92|
|       1029|      621.5|
|       1032|     862.59|
|       1010|     775.43|
|       1002|      72.91|
|       1048|      662.2|
|       1050|     238.84|
|       1035|     373.01|
|       1045|     274.26|
|       1017|    1537.29|
|       1037|     498.38|
|       1036|     113.78|
+-----------+-----------+
only showing top 20 rows



In [0]:
# Ordena a coluna total_gasto do maior valor para o menor
df_total_gasto.orderBy("total_gasto", ascending=False).show()

+-----------+-----------+
|customer_id|total_gasto|
+-----------+-----------+
|       1043|    1561.97|
|       1017|    1537.29|
|       1046|    1420.77|
|       1041|    1195.31|
|       1040|    1022.37|
|       1027|    1019.13|
|       1038|    1009.35|
|       1014|     940.46|
|       1032|     862.59|
|       1024|      848.8|
|       1016|     809.11|
|       1004|     804.11|
|       1003|     788.37|
|       1031|     787.43|
|       1010|     775.43|
|       1007|     756.29|
|       1025|     734.46|
|       1034|     715.98|
|       1049|      711.7|
|       1020|     662.53|
+-----------+-----------+
only showing top 20 rows



In [0]:
# Identifica os 3 clientes que mais gastaram 
df_total_gasto.orderBy("total_gasto", ascending=False).limit(3).show()

+-----------+-----------+
|customer_id|total_gasto|
+-----------+-----------+
|       1043|    1561.97|
|       1017|    1537.29|
|       1046|    1420.77|
+-----------+-----------+

